In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import wrds
# import data_read
import pull_WRDS_call_reports, pull_treasuries_data
import pull_mbb_data
import data_preprocessing, compute_treasury_changes
# import calc_functions
# import Calc_table_statistic
import helper_functions

warnings.filterwarnings("ignore")

In [2]:
import importlib
importlib.reload(pull_WRDS_call_reports)
importlib.reload(data_preprocessing)

<module 'data_preprocessing' from '/Users/aadi/projects/svb-failure-case-study/src/data_preprocessing.py'>

### Pull and Preprocess WRDS Call Research Data

In [3]:
df = pull_WRDS_call_reports.load_wrds_call_research()

In [4]:
df.shape

(24565, 371)

In [5]:
df.dtypes

rssd9001                        object
rcon9804                       float64
date                    datetime64[ns]
rssd9050                        object
rssd9055                        object
                             ...      
avgirate_timedep                object
avgirate_timedep_ytd            object
avgirate_savdep                 object
avgirate_fordep                 object
fedfundsrate                   float64
Length: 371, dtype: object

In [6]:
df_filtered = df
df_filtered.head()

,rssd9001,rcon9804,date,rssd9050,rssd9055,rssd9048,assets,cash,securities,securities_asu,...,q_intexptradingandborrowed,q_intincpersloans,q_persloansintinc,q_intincreloans,q_intexpdomdep,avgirate_timedep,avgirate_timedep_ytd,avgirate_savdep,avgirate_fordep,fedfundsrate
271,37,59.0,2022-03-31,10057,0,200.0,87842.0,12640.0,53347.0,53347.0,...,NaN,131.0,131.0,157.0,55.0,0.0079,0.0079,0.0017,None,0.0018
273,37,59.0,2022-06-30,10057,0,200.0,84231.0,8061.0,51665.0,51665.0,...,NaN,135.0,135.0,153.0,53.0,0.0076,0.0080,0.0017,None,0.0113
274,37,59.0,2022-09-30,10057,0,200.0,80081.0,4751.0,49439.0,49439.0,...,NaN,141.0,141.0,153.0,53.0,0.0079,0.0081,0.0016,None,0.0252
275,37,59.0,2022-12-31,10057,0,200.0,81497.0,9260.0,50548.0,50548.0,...,NaN,155.0,155.0,154.0,52.0,0.0080,0.0083,0.0017,None,0.0408
277,37,59.0,2023-03-31,10057,0,200.0,80558.0,5977.0,50973.0,50973.0,...,NaN,146.0,146.0,147.0,75.0,0.0117,0.0118,0.0024,None,0.0463


In [7]:
df_filtered = df_filtered.sort_values(by=['date'])  # Ensure the data is sorted by date
df_filtered = df_filtered.groupby('rssd9001').first().reset_index()

In [8]:
# df_filtered[df_filtered['rssd9001'] == '37'].head()

In [9]:
df_filtered.shape

(4913, 371)

In [10]:
len(df_filtered['rssd9001'].unique())

4913

In [11]:
df_new = df_filtered[['rssd9001', 'date', 'assets', 'securitiesheldtomaturity',
                      'securitiesavailableforsale', 'mbsassets', 'absassets', 'loans', 'totaldep', 'alldepuninsured', 'treasurysec', 'timedepuninsured', 'domdepuninsured',
                      'securitiesrmbs_less_3m', 'securitiesrmbs_3m_1y', 'securitiesrmbs_1y_3y', 'securitiesrmbs_3y_5y', 'securitiesrmbs_5y_15y', 'securitiesrmbs_over_15y',
                      'resloans_less_3m', 'resloans_3m_1y', 'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y', 'resloans_over_15y']]

In [12]:
df_new.shape

(4913, 25)

### Load and Preprocess ETF MBS Data

In [13]:
etf_mbs = pull_mbb_data.load_from_manual_csv()

✅ Successfully loaded MBB data from /Users/aadi/projects/svb-failure-case-study/data/manual/ishares_mbs_etf_daily.csv


In [14]:
etf_mbs.head()

,Date,Close/Last,Volume,Open,High,Low
0,02/25/2025,93.54,2839593,93.43,93.6276,93.060
1,02/24/2025,92.99,2637865,92.64,93.0600,92.550
2,02/21/2025,92.87,1584830,92.52,92.9750,92.415
3,02/20/2025,92.38,1627928,92.26,92.4299,92.250
4,02/19/2025,92.13,2126798,91.91,92.1872,91.870


In [15]:
type(etf_mbs.iloc[0]['Date'])

str

### Load and Preprocess Treasuries Data

In [16]:
treasuries_data = pull_treasuries_data.load_from_manual_excel()

✅ Successfully loaded Treasury Bond Index data from /Users/aadi/projects/svb-failure-case-study/data/manual/s_&_p_treasury_bond_index.xls


In [17]:
treasuries_data.head()

,Effective date,S&P U.S. Treasury Bond Index
0,2015-01-30,414.51
1,2015-02-02,414.20
2,2015-02-03,412.52
3,2015-02-04,412.79
4,2015-02-05,412.07


In [18]:
treasuries_data = data_preprocessing.preprocess_treasuries_data(treasuries_data)

In [19]:
treasuries_data.head()

,date,index
0,2015-01-30,414.51
1,2015-02-02,414.20
2,2015-02-03,412.52
3,2015-02-04,412.79
4,2015-02-05,412.07


In [20]:
df_new.head()

,rssd9001,date,assets,securitiesheldtomaturity,securitiesavailableforsale,mbsassets,absassets,loans,totaldep,alldepuninsured,...,securitiesrmbs_1y_3y,securitiesrmbs_3y_5y,securitiesrmbs_5y_15y,securitiesrmbs_over_15y,resloans_less_3m,resloans_3m_1y,resloans_1y_3y,resloans_3y_5y,resloans_5y_15y,resloans_over_15y
0,1000052,2022-03-31,563110.0,9155.0,115340.0,92960.0,0.0,377073.0,473143.0,115688.0,...,4.0,244.0,4915.0,0.0,3534.0,6017.0,10089.0,4826.0,3743.0,4574.0
1,1000276,2022-03-31,399377.0,0.0,99292.0,46426.0,0.0,249412.0,336076.0,95359.0,...,0.0,0.0,12487.0,23889.0,637.0,5082.0,13259.0,20434.0,47627.0,56174.0
2,1000641,2022-03-31,790350.0,226882.0,98841.0,186588.0,0.0,362852.0,718769.0,237702.0,...,337.0,1589.0,92159.0,30708.0,995.0,1154.0,1331.0,1610.0,28272.0,75531.0
3,1000856,2022-03-31,28661.0,0.0,7794.0,0.0,0.0,11580.0,22407.0,2263.0,...,0.0,0.0,0.0,0.0,422.0,305.0,2173.0,3575.0,573.0,0.0
4,1000959,2022-03-31,213578.0,0.0,76173.0,32918.0,180.0,99876.0,192585.0,53904.0,...,998.0,0.0,13107.0,0.0,587.0,3327.0,6898.0,7978.0,924.0,0.0


In [21]:
df_sorted = df_new.sort_values(by=['date'])
df_final = df_sorted.groupby('rssd9001').first().reset_index()

### Calculate MTM Losses

In [22]:
etf_mbs_change = helper_functions.get_etf_mbs_change(etf_mbs)
treasury_change = helper_functions.get_treasury_change(treasuries_data)

In [23]:
df_final['mtm_loss_mbs'] = df_final['mbsassets'] * etf_mbs_change
df_final['mtm_loss_treasury'] = df_final['treasurysec'] * treasury_change
df_final['mtm_loss_loans'] = df_final['loans'] * treasury_change
df_final['total_mtm_loss'] = df_final['mtm_loss_mbs'] + df_final['mtm_loss_treasury'] + df_final['mtm_loss_loans']

In [24]:
aggregate_loss = df_final['total_mtm_loss'].sum()

In [25]:
df_final['share_rmbs'] = df_final['mtm_loss_mbs'] / df_final['total_mtm_loss']
df_final['share_treasury_other'] = df_final['mtm_loss_treasury'] / df_final['total_mtm_loss']
df_final['share_residential_mortgage'] = df_final['mtm_loss_loans'] / df_final['total_mtm_loss']

In [26]:
df_final['loss_to_asset'] = df_final['total_mtm_loss'] / df_final['assets']
df_final['uninsured_deposit_mm_asset'] = df_final['alldepuninsured'] / (df_final['assets'] - df_final['total_mtm_loss'])

### Assets

In [27]:
df_assets = df_new[['rssd9001', 'date', 'assets']]

In [28]:
df_asset = df_assets #total assets all banks

#list of GSIB bank IDs
GSIB = helper_functions.GSIB_bank_id()
GSIB = [str(element) for element in GSIB]

#total assets all GSIB banks
df_asset_GSIB = df_asset[df_asset['rssd9001'].isin(GSIB)]

#total assets large non-GSIB banks
df_asset_large_ex_GSIB = df_asset[(~df_asset['rssd9001'].isin(GSIB)) & (df_asset['assets']>1384000)]

#total assets small banks
df_asset_small = df_asset[(~df_asset['rssd9001'].isin(GSIB)) & (df_asset['assets']<=1384000)] 

In [29]:
df.dtypes

rssd9001                        object
rcon9804                       float64
date                    datetime64[ns]
rssd9050                        object
rssd9055                        object
                             ...      
avgirate_timedep                object
avgirate_timedep_ytd            object
avgirate_savdep                 object
avgirate_fordep                 object
fedfundsrate                   float64
Length: 371, dtype: object

In [30]:
df_asset[df_asset['rssd9001'] == '30810'].head()

,rssd9001,date,assets
1475,30810,2022-03-31,105657408.0


In [31]:
len(df_asset_large_ex_GSIB), len(df_asset_GSIB), len(df_asset_small), len(df_new)

(831, 17, 4064, 4913)

### Insured

In [32]:
df_insured = df_new[['rssd9001', 'date', 'alldepuninsured','timedepuninsured', 'domdepuninsured']]

### RMBs

In [33]:
df_rmbs = df_final[['rssd9001', 'date', 'securitiesrmbs_less_3m', 'securitiesrmbs_3m_1y', 'securitiesrmbs_1y_3y', 'securitiesrmbs_3y_5y', 'securitiesrmbs_5y_15y', 'securitiesrmbs_over_15y']]

In [34]:
df_rmbs[df_rmbs['rssd9001'] == '37'].head()

,rssd9001,date,securitiesrmbs_less_3m,securitiesrmbs_3m_1y,securitiesrmbs_1y_3y,securitiesrmbs_3y_5y,securitiesrmbs_5y_15y,securitiesrmbs_over_15y
2037,37,2022-03-31,0.0,0.0,0.0,0.0,856.0,6021.0


### Loans

In [35]:
df_loans = df_final[['rssd9001', 'date', 'resloans_less_3m', 'resloans_3m_1y', 'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y', 'resloans_over_15y']]

In [36]:
df_loans[df_loans['rssd9001'] == '37'].head()

,rssd9001,date,resloans_less_3m,resloans_3m_1y,resloans_1y_3y,resloans_3y_5y,resloans_5y_15y,resloans_over_15y
2037,37,2022-03-31,355.0,354.0,2430.0,2229.0,2641.0,0.0


In [37]:
df_final.columns

Index(['rssd9001', 'date', 'assets', 'securitiesheldtomaturity',
       'securitiesavailableforsale', 'mbsassets', 'absassets', 'loans',
       'totaldep', 'alldepuninsured', 'treasurysec', 'timedepuninsured',
       'domdepuninsured', 'securitiesrmbs_less_3m', 'securitiesrmbs_3m_1y',
       'securitiesrmbs_1y_3y', 'securitiesrmbs_3y_5y', 'securitiesrmbs_5y_15y',
       'securitiesrmbs_over_15y', 'resloans_less_3m', 'resloans_3m_1y',
       'resloans_1y_3y', 'resloans_3y_5y', 'resloans_5y_15y',
       'resloans_over_15y', 'mtm_loss_mbs', 'mtm_loss_treasury',
       'mtm_loss_loans', 'total_mtm_loss', 'share_rmbs',
       'share_treasury_other', 'share_residential_mortgage', 'loss_to_asset',
       'uninsured_deposit_mm_asset'],
      dtype='object')

### Treasuries

In [38]:
df_treasuries = df_filtered[['rssd9001', 'date', 'securitiestreasury_less_3m',
'securitiestreasury_3m_1y',
'securitiestreasury_1y_3y',
'securitiestreasury_3y_5y',
'securitiestreasury_5y_15y',
'securitiestreasury_over_15y']]

In [39]:
df_treasuries[df_treasuries['rssd9001'] == '37'].head()

,rssd9001,date,securitiestreasury_less_3m,securitiestreasury_3m_1y,securitiestreasury_1y_3y,securitiestreasury_3y_5y,securitiestreasury_5y_15y,securitiestreasury_over_15y
2037,37,2022-03-31,0.0,0.0,330.0,708.0,26220.0,19212.0


### Other Loans

In [40]:
df_other_loans = df_filtered[['rssd9001', 'date', 'loansleases_less_3m',
'loansleases_3m_1y',
'loansleases_1y_3y',
'loansleases_3y_5y',
'loansleases_5y_15y',
'loansleases_over_15y']]

In [41]:
df_other_loans[df_other_loans['rssd9001'] == '37'].head()

,rssd9001,date,loansleases_less_3m,loansleases_3m_1y,loansleases_1y_3y,loansleases_3y_5y,loansleases_5y_15y,loansleases_over_15y
2037,37,2022-03-31,1437.0,3928.0,5358.0,5317.0,4747.0,0.0


### Other

In [42]:
df_rmbs = df_rmbs.rename(columns={
'securitiesrmbs_less_3m': '<3m',
'securitiesrmbs_3m_1y':'3m-1y',
'securitiesrmbs_1y_3y':'1y-3y',
'securitiesrmbs_3y_5y':'3y-5y',
'securitiesrmbs_5y_15y':'5y-15y',
'securitiesrmbs_over_15y':'>15y',
})

df_treasuries = df_treasuries.rename(columns={
'securitiestreasury_less_3m': '<3m',
'securitiestreasury_3m_1y':'3m-1y',
'securitiestreasury_1y_3y':'1y-3y',
'securitiestreasury_3y_5y':'3y-5y',
'securitiestreasury_5y_15y':'5y-15y',
'securitiestreasury_over_15y':'>15y',
})

df_loans = df_loans.rename(columns={
'resloans_less_3m': '<3m',
'resloans_3m_1y':'3m-1y',
'resloans_1y_3y':'1y-3y',
'resloans_3y_5y':'3y-5y',
'resloans_5y_15y':'5y-15y',
'resloans_over_15y':'>15y',
})

df_other_loans = df_other_loans.rename(columns={
'loansleases_less_3m': '<3m',
'loansleases_3m_1y':'3m-1y',
'loansleases_1y_3y':'1y-3y',
'loansleases_3y_5y':'3y-5y',
'loansleases_5y_15y':'5y-15y',
'loansleases_over_15y':'>15y',
})

In [43]:
sum_asset = 0
for df in [df_rmbs, df_loans, df_treasuries, df_other_loans]:
    total = df[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
    sum_asset += total

print('total assets:',"{:,.2f}".format(sum_asset))
total_asset = df_asset['assets'].sum()
print('assets ratio:',"{:,.2%}".format(sum_asset/total_asset))

total assets: 18,174,670,246.00
assets ratio: 65.92%


In [44]:
rcon_series_1 = pull_WRDS_call_reports.load_RCON_series_1()

In [45]:
rcon_series_1['rssd9999'] = pd.to_datetime(rcon_series_1['rssd9999'])
# Define required dates
required_dates = [pd.Timestamp("2022-03-31"), pd.Timestamp("2023-03-31")]

rcon_series_1 = rcon_series_1.groupby('rssd9001').filter(lambda x: all(date in x['rssd9999'].values for date in required_dates))
rcon_series_1 = rcon_series_1.rename(columns={
    'rssd9999': 'date'
})

In [46]:
df_uninsured = rcon_series_1[['rssd9001', 'date', 'uninsured_deposits']]
df_insured = rcon_series_1[['rssd9001', 'date', 'insured_deposit_1', 'insured_deposit_2']]

In [47]:
asset_small = df_asset_small['rssd9001'].unique()
asset_large_ex_GSIB = df_asset_large_ex_GSIB['rssd9001'].unique()

In [48]:
len(df_asset_large_ex_GSIB), len(df_asset_small), len(GSIB)

(831, 4064, 17)

In [49]:
importlib.reload(helper_functions)
etf_mbs_change = helper_functions.get_etf_mbs_change(etf_mbs)
treasury_change = helper_functions.get_treasury_change(treasuries_data)
rmbs_multiplier = etf_mbs_change / treasury_change
rmbs_multiplier

1.8904405040465027

In [50]:
etf_mbs.tail()

,Date,Close/Last,Volume,Open,High,Low
1252,03/03/2020,109.81,4320449,109.58,109.8900,109.520
1253,03/02/2020,109.55,2277338,109.61,109.7500,109.465
1254,02/28/2020,109.50,2779204,109.53,109.7700,109.474
1255,02/27/2020,109.28,1642608,109.31,109.4000,109.230
1256,02/26/2020,109.16,2021247,109.16,109.2501,109.080


In [51]:
rmbs_multiplier

1.8904405040465027

In [52]:
treasury_prices = pd.read_excel("../data/manual/combined_index_df.xlsx")
treasury_prices.set_index('date', inplace=True)
start_date = "2022-03-31"
end_date ="2023-03-31"

In [53]:
importlib.reload(helper_functions)
bank_losses = helper_functions.calculate_losses(treasury_prices, df_rmbs, df_loans, df_treasuries, df_other_loans, df_asset, rmbs_multiplier)

In [54]:
bank_losses[bank_losses['bank_ID'] == '12311'].head()

,date,bank_ID,RMBs_loss,treasury_loss,loans_loss,other_loan_loss,total_loss,Share RMBs,Share Treasury and Other,Share Residential Mortgage,Share Other Loan,RMBs_asset,treasury_asset,residential_mortgage_asset,other_loan_asset,core_asset,gross_asset,loss/core_asset,loss/gross_asset
231,2022-03-31,12311,-7.668946e+06,-352863.839663,-4.806793e+06,-3.169639e+06,-1.599824e+07,47.93618,2.205641,30.045756,19.812423,23910558.0,6623679.0,22529389.0,114651981.0,167715607.0,176020595.0,0.095389,0.090888


In [55]:
median_percentage = bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].median()
median_percentage

Share RMBs                     9.427995
Share Treasury and Other      10.008172
Share Residential Mortgage    20.949797
Share Other Loan              33.522608
dtype: float64

In [56]:
std_percentages =bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].std()
std_percentages

Share RMBs                     83.869974
Share Treasury and Other      104.999900
Share Residential Mortgage     45.973768
Share Other Loan              153.570322
dtype: float64

In [57]:
total_sum_loss = bank_losses['total_loss'].sum()
total_sum_loss

-1550070097.810647

In [58]:
median_bank_loss = bank_losses['total_loss'].median()
median_bank_loss 

-14980.472235690078

In [59]:
core_asset = bank_losses['core_asset'].sum()
gross_asset = bank_losses['gross_asset'].sum()
core_asset / gross_asset

0.659237084971283

In [60]:
median_loss_asset_ratio = bank_losses['loss/gross_asset'].median()
median_loss_asset_ratio

0.05037982996684826

In [61]:
average_loss_asset_ratio = bank_losses['loss/gross_asset'].mean()
average_loss_asset_ratio

0.06346668599131544

In [62]:
std_loss_asset_ratio = bank_losses['loss/gross_asset'].std()
std_loss_asset_ratio

0.057407785426781016

In [63]:
df_uninsured.head()
uninsured_deposit = df_uninsured[df_uninsured['date'] == '2022-03-31']

In [64]:
uninsured_deposit['rssd9001'] = uninsured_deposit['rssd9001'].astype(str)

In [65]:
uninsured_deposit.head()

,rssd9001,date,uninsured_deposits
2,37,2022-03-31,NaN
8,21256,2022-03-31,NaN
10,457,2022-03-31,NaN
12,242,2022-03-31,NaN
26,279,2022-03-31,NaN


In [66]:
uninsured_deposit['uninsured_deposits'].sum()/gross_asset 

0.29453071112953516

In [67]:
uninsured_deposit['uninsured_deposits'].sum()

8119990022.0

In [68]:
uninsured_deposit.columns

Index(['rssd9001', 'date', 'uninsured_deposits'], dtype='object')

In [69]:
importlib.reload(helper_functions)
un_mm_ratio = helper_functions.calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses)

In [70]:
un_mm_ratio['total_loss'].sum()

-1550070097.810647

In [71]:
un_mm_ratio['total_asset'].sum()

27569247332.0

In [72]:
un_mm_ratio['mm_asset'].sum()

26019177234.189354

In [73]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].median()

0.27348882382991996

In [74]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].std()

0.19243917613060021

In [75]:
#df_insured['insured_deposit'] = df_insured['insured_deposit_1'] + df_insured['insured_deposit_2']

In [76]:
# df_insured['insured_deposit'].sum()

In [77]:
importlib.reload(helper_functions)
large_ex_GSIB = helper_functions.large_ex_GSIB_bank_id(df_asset_large_ex_GSIB)
small = helper_functions.small_bank_id(df_asset_small)

In [78]:
len(df_asset_GSIB), len(df_asset_large_ex_GSIB), len(df_asset_small)

(17, 831, 4064)

In [79]:
df_RMBS_GSIB = df_rmbs[df_rmbs['rssd9001'].isin(GSIB)]
df_RMBS_large_ex_GSIB = df_rmbs[df_rmbs['rssd9001'].isin(large_ex_GSIB)]
df_RMBS_small = df_rmbs[df_rmbs['rssd9001'].isin(small)]
len(df_RMBS_GSIB), len(df_RMBS_large_ex_GSIB), len(df_RMBS_small)

(17, 831, 4064)

In [80]:
df_loans_GSIB = df_loans[df_loans['rssd9001'].isin(GSIB)]
df_loans_large_ex_GSIB = df_loans[df_loans['rssd9001'].isin(large_ex_GSIB)]
df_loans_small = df_loans[df_loans['rssd9001'].isin(small)]
len(df_loans_GSIB), len(df_loans_large_ex_GSIB), len(df_loans_small)

(17, 831, 4064)

In [81]:
df_treasuries_GSIB = df_treasuries[df_treasuries['rssd9001'].isin(GSIB)]
df_treasuries_large_ex_GSIB = df_treasuries[df_treasuries['rssd9001'].isin(large_ex_GSIB)]
df_treasuries_small = df_treasuries[df_treasuries['rssd9001'].isin(small)]
len(df_treasuries_GSIB), len(df_treasuries_large_ex_GSIB), len(df_treasuries_small)

(17, 831, 4064)

In [82]:
df_other_loans_GSIB = df_other_loans[df_other_loans['rssd9001'].isin(GSIB)]
df_other_loans_large_ex_GSIB = df_other_loans[df_other_loans['rssd9001'].isin(large_ex_GSIB)]
df_other_loans_small = df_other_loans[df_other_loans['rssd9001'].isin(small)]
len(df_other_loans_GSIB), len(df_other_loans_large_ex_GSIB), len(df_other_loans_small)

(17, 831, 4064)

In [83]:
uninsured_deposit_GSIB = uninsured_deposit[uninsured_deposit['rssd9001'].isin(GSIB)]
uninsured_deposit_large_ex_GSIB = uninsured_deposit[uninsured_deposit['rssd9001'].isin(large_ex_GSIB)]
uninsured_deposit_small = uninsured_deposit[uninsured_deposit['rssd9001'].isin(small)]
len(uninsured_deposit_GSIB), len(uninsured_deposit_large_ex_GSIB), len(uninsured_deposit_small)

(17, 712, 3976)

In [84]:
bank_losses = helper_functions.calculate_losses(
    treasury_prices, df_rmbs, df_loans, df_treasuries, df_other_loans, df_asset, rmbs_multiplier
)

In [85]:
bank_losses_gsib = helper_functions.calculate_losses(
    treasury_prices, df_RMBS_GSIB, df_loans_GSIB, df_treasuries_GSIB, df_other_loans_GSIB, df_asset_GSIB, rmbs_multiplier 
)

In [86]:
bank_losses_large = helper_functions.calculate_losses(
    treasury_prices, df_RMBS_large_ex_GSIB, df_loans_large_ex_GSIB, df_treasuries_large_ex_GSIB, df_other_loans_large_ex_GSIB, df_asset_large_ex_GSIB, rmbs_multiplier
)

In [87]:
bank_losses_small = helper_functions.calculate_losses(
    treasury_prices, df_RMBS_small, df_loans_small, df_treasuries_small, df_other_loans_small, df_asset_large_ex_GSIB, rmbs_multiplier
)

In [88]:
uninsured_deposit_mm_asset_small = helper_functions.calculate_uninsured_deposit_mm_asset(uninsured_deposit_small, bank_losses_small)
uninsured_deposit_mm_asset_large_ex_GSIB = helper_functions.calculate_uninsured_deposit_mm_asset(uninsured_deposit_large_ex_GSIB, bank_losses_large)
uninsured_deposit_mm_asset_GSIB = helper_functions.calculate_uninsured_deposit_mm_asset(uninsured_deposit_GSIB, bank_losses_gsib)    

In [89]:
importlib.reload(helper_functions)
final_stats = helper_functions.final_statistic_table(bank_losses, un_mm_ratio, None)
final_stats_small = helper_functions.final_statistic_table(bank_losses_small, uninsured_deposit_mm_asset_small, None, index_name = 'Small Banks')
final_stats_large_ex_GSIB = helper_functions.final_statistic_table(bank_losses_large, uninsured_deposit_mm_asset_large_ex_GSIB, None, index_name = 'Large Ex GSIB Banks')
final_stats_GSIB = helper_functions.final_statistic_table(bank_losses_gsib, uninsured_deposit_mm_asset_GSIB, None, index_name = 'GSIB Banks')

In [90]:
table_1 = pd.concat([final_stats, final_stats_small, final_stats_large_ex_GSIB, final_stats_GSIB], axis=1)
table_1

,All Banks,Small Banks,Large Ex GSIB Banks,GSIB Banks
Aggregate Loss,1.6T,-0.0T,0.7T,0.7T
Bank Level Loss,15.0M,-0.0M,151.5M,6905.9M
Bank Level Loss Std,5.57B,0.0B,3.92B,81.91B
Share RMBS,942.8,0.0,1257.1,3450.2
Share RMBS Std,8387.0,0.0,5012.2,3853.6
Share Treasury and Other,1000.8,0.0,508.0,757.0
Share Treasury and Other Std,10500.0,0.0,2486.2,818.8
Share Residential Mortgage,2095.0,0.0,2221.3,2003.3
Share Residential Mortgage Std,4597.4,0.0,3376.7,2925.4
Share Other Loan,3352.3,0.0,3161.8,1795.5


In [91]:
helper_functions.save_dataframe_as_latex_table_1(table_1, filename="replicated_table_1.tex", caption="Replicated Table 1: Mark-to-Market Statistics by Bank Size", label="tab:replicated1")

LaTeX table saved as replicated_table_1.tex


In [92]:
table_1.index

Index(['Aggregate Loss', 'Bank Level Loss', 'Bank Level Loss Std',
       'Share RMBS', 'Share RMBS Std', 'Share Treasury and Other',
       'Share Treasury and Other Std', 'Share Residential Mortgage',
       'Share Residential Mortgage Std', 'Share Other Loan',
       'Share Other Loan Std', 'Loss/Asset', 'Loss/Asset Std',
       'Uninsured Deposit/MM Asset', 'Uninsured Deposit/MM Asset Std',
       'Number of Banks'],
      dtype='object')